In [1]:
import parallization_metacentrum as pm

# Normal Jobs
- splits the work across number of cores

In [2]:
# define jobs ...
allinp = list(range(40))

def funkce(inp):
    return inp * 2

dispatcher = pm.IsMetacentrum()

In [3]:
# returns number of cpus (local == number of cores, metacentrum == number of assigned cores)
print(dispatcher.cpus)

16


In [4]:
# returns GB (local == number of cores, metacentrum == assigned memory)
print(dispatcher.memory)

38.6578369140625


In [5]:
# 
dispatcher.ismetacentrum

False

In [6]:
# run parallel (d.cpus) list.map
print(dispatcher.run_map(funkce, allinp))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]


In [7]:
# lazy version
print(dispatcher.run_imap(funkce, allinp))

<generator object IsMetacentrum.run_imap at 0x7fa196cf73e0>


# Array jobs 
- split work across array
- works in normal jobs too

In [8]:

# hack ... simulate metacentrum environment
class HackArray(pm.ArrayMetacentrum):
    @property
    def array_index(self):
        return 1 # <- 0, 1, 2 
    
    def _get_all_array_indexes(self):
        return [1,2,3]

# slice_type='interleave' - default
array = HackArray()

# define jobs ...
allinp = list(range(40))

def funkce(inp):
    return inp * 2

In [9]:
# just split the work based on index
array.split_work(allinp)

In [10]:
# apply function on data and return list of answers
array.split_work_and_run_map(funkce, allinp)

[2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 74]

In [11]:
# lazy version ... you can use tqdm (progress bar)
h = array.split_work_and_run_imap(funkce, allinp)
print(h)
print('-'*40)
for g in h:
    print(g, end=', ')
print()

<generator object ArrayMetacentrum.split_work_and_run_imap at 0x7fa196cf76f0>
----------------------------------------
2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 74, 


In [12]:
# even lazier version (unordered) ... you can use tqdm (progress bar)
h = array.split_work_and_run_imap_unordered(funkce, allinp)
print(h)
print('-'*40)
for g in h:
    print(g, end=', ')
print()

<generator object ArrayMetacentrum.split_work_and_run_imap_unordered at 0x7fa196cf7840>
----------------------------------------
2, 8, 14, 20, 26, 44, 32, 38, 56, 50, 62, 68, 74, 


## Slice types

In [13]:
# simulates array ...

class HackArray0(HackArray):
    @property
    def array_index(self): return 0 
class HackArray1(HackArray):
    @property
    def array_index(self): return 1 
class HackArray2(HackArray):
    @property
    def array_index(self): return 2 
    
allinp = list(range(40))

In [19]:
# every #arrayElement is selected
array = HackArray0(slice_type='interleave')
print(array.split_work(allinp))
array = HackArray1(slice_type='interleave')
print(array.split_work(allinp))
array = HackArray2(slice_type='interleave')
print(array.split_work(allinp))

In [15]:
# selects only continuous parts of iterator
array = HackArray0(slice_type='offset')
print(array.split_work(allinp))
array = HackArray1(slice_type='offset')
print(array.split_work(allinp))
array = HackArray2(slice_type='offset')
print(array.split_work(allinp))

[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [16]:
# random shuffle + split
array = HackArray0(slice_type='random')
print(array.split_work(allinp))
array = HackArray1(slice_type='random')
print(array.split_work(allinp))
array = HackArray2(slice_type='random')
print(array.split_work(allinp))

[33, 2, 0, 38, 27, 5, 6, 8, 14, 15, 17, 1, 7]
[12, 34, 35, 26, 13, 22, 20, 31, 37, 29, 19, 16, 39]
[9, 3, 10, 36, 25, 24, 30, 11, 4, 32, 21, 28, 23, 18]


In [17]:
# random shuffle + split
array = HackArray0(slice_type='random', seed=1)
print(array.split_work(allinp))
array = HackArray1(slice_type='random', seed=1)
print(array.split_work(allinp))
array = HackArray2(slice_type='random', seed=1)
print(array.split_work(allinp))

[15, 3, 6, 25, 24, 30, 28, 31, 7, 16, 4, 36, 8]
[27, 22, 10, 33, 18, 35, 39, 14, 26, 19, 13, 12, 0]
[23, 2, 11, 1, 37, 32, 5, 9, 17, 29, 34, 20, 38, 21]


In [18]:
# random shuffle + split
array = HackArray0(slice_type='random', seed=2)
print(array.split_work(allinp))
array = HackArray1(slice_type='random', seed=2)
print(array.split_work(allinp))
array = HackArray2(slice_type='random', seed=2)
print(array.split_work(allinp))

[32, 37, 21, 18, 2, 13, 16, 19, 10, 23, 38, 5, 3]
[7, 22, 0, 1, 8, 24, 14, 17, 11, 33, 36, 12, 20]
[26, 4, 39, 27, 34, 15, 29, 9, 35, 31, 30, 25, 6, 28]
